# Data Preprocessing :  Used Cars Dataset 


Link for the download of Used cars dataset: https://www.kaggle.com/adityadesai13/used-car-dataset-ford-and-mercedes
and extract the content into the `fedbiomed/notebooks/data` folder

In [ ]:
# change here with the directory where you downloaded the dataset
data_dir = './data/used_cars'

In [ ]:
import pandas as pd

# Dataset Details
The data consists of used cars listings. 100,000 listings, which have been separated into files corresponding to each car manufacturer. Each file will simulate data for each node.

# Goal

The goal of this tutorial is to build a federated regression model on Non-IID dataset and generate the best model by performing validation on hold out dataset and tuning hyperparameters.The metric used to decide best model is RMSE.

In [ ]:
import os

# Use audi and bmw for training on 2 nodes
audi = pd.read_csv(os.path.join(data_dir, "audi.csv"))
bmw = pd.read_csv(os.path.join(data_dir, "bmw.csv"))

# Use Ford for final validation at central researcher (test dataset)
ford = pd.read_csv(os.path.join(data_dir, "ford.csv"))

# Use the following csvs if you want to run more than 2 nodes. Uncomment Corresponding lines in the following cell blocks
# cclass = pd.read_csv(os.path.join(data_dir, "cclass.csv"))
# focus = pd.read_csv(os.path.join(data_dir, "focus.csv"))
# hyundai = pd.read_csv(os.path.join("data_dir, "huyndai.csv"))
# merc = pd.read_csv(os.path.join(data_dir, "merc.csv"))
# skoda = pd.read_csv(os.path.join(data_dir, "skoda.csv"))
# toyota = pd.read_csv(os.path.join(data_dir, "toyota.csv"))
# vauxhall = pd.read_csv(os.path.join(data_dir, "vauxhall.csv"))
# vw = pd.read_csv(os.path.join(data_dir, "vw.csv"))

Drop columns model & fuelType as labels are not consistent across files. 

In [ ]:
audi.drop(columns = ['model','fuelType'],inplace = True)
bmw.drop(columns = ['model','fuelType'],inplace = True)
ford.drop(columns = ['model','fuelType'],inplace = True)

# cclass.drop(columns = ['model','fuelType'],inplace = True)
# focus.drop(columns = ['model','fuelType'],inplace = True)
# hyundai.drop(columns = ['model','fuelType'],inplace = True)
# merc.drop(columns = ['model','fuelType'],inplace = True)
# skoda.drop(columns = ['model','fuelType'],inplace = True)
# toyata.drop(columns = ['model','fuelType'],inplace = True)
# vauxhall.drop(columns = ['model','fuelType'],inplace = True)
# vw.drop(columns = ['model','fuelType'],inplace = True)

Label encode transmission column

In [ ]:
audi['transmission'] = audi['transmission'].map({'Automatic':0,'Manual':1,'Semi-Auto':2,'Other':3})
bmw['transmission'] = bmw['transmission'].map({'Automatic':0,'Manual':1,'Semi-Auto':2,'Other':3})
ford['transmission'] = ford['transmission'].map({'Automatic':0,'Manual':1,'Semi-Auto':2,'Other':3})

# cclass['transmission'] = cclass['transmission'].map({'Automatic':0,'Manual':1,'Semi-Auto':2,'Other':3})
# focus['transmission'] = focus['transmission'].map({'Automatic':0,'Manual':1,'Semi-Auto':2,'Other':3})
# hyundai['transmission'] = hyundai['transmission'].map({'Automatic':0,'Manual':1,'Semi-Auto':2,'Other':3})
# merc['transmission'] = merc['transmission'].map({'Automatic':0,'Manual':1,'Semi-Auto':2,'Other':3})
# skoda['transmission'] = skoda['transmission'].map({'Automatic':0,'Manual':1,'Semi-Auto':2,'Other':3})
# toyata['transmission'] = toyata['transmission'].map({'Automatic':0,'Manual':1,'Semi-Auto':2,'Other':3})
# vauxhall['transmission'] = vauxhall['transmission'].map({'Automatic':0,'Manual':1,'Semi-Auto':2,'Other':3})
# vw['transmission'] = vw['transmission'].map({'Automatic':0,'Manual':1,'Semi-Auto':2,'Other':3})

In [ ]:
audi.to_csv(os.path.join(data_dir, 'audi_transformed.csv'),header = True,index= False)
bmw.to_csv(os.path.join(data_dir, 'bmw_transformed.csv'),header = True,index= False)
ford.to_csv(os.path.join(data_dir, 'ford_transformed.csv'),header = True,index= False)

# cclass.to_csv('cclass_transformed.csv',header = True,index= False)
# focus.to_csv('focus_transformed.csv',header = True,index= False)
# hyundai.to_csv('huydai_transformed.csv',header = True,index= False)
# merc.to_csv('merc_transformed.csv',header = True,index= False)
# skoda.to_csv('skoda_transformed.csv',header = True,index= False)
# toyata.to_csv('toyata_transformed.csv',header = True,index= False)
# vauxhall.to_csv('vaxhall_transformed.csv',header = True,index= False)
# vw.to_csv('vw_transformed.csv',header = True,index= False)

# Fed-BioMed Researcher to train a model on a Used Cars dataset

## Start the network
Before running this notebook, start the network with `./scripts/fedbiomed_run network`

## Setting the nodes up
It is necessary to previously configure 2 nodes:
1. `./scripts/fedbiomed_run node config config1.ini add` (node1) and `./scripts/fedbiomed_run node config config2.ini add` (node2)
  * Select option 1 to add a csv file to the node
    * use the `audi_transformed.csv` file (node 1) and `bmw_transformed.csv` file (node 2)
  * Choose the name, tags and description of the dataset
    * choose tag `UsedCars` (or modify the used tag in this notebook)
  * Spin as many nodes as you want(max nodes 11 for 11 csv files in used cars dataset). Hold out one file for testing.
  * Load the .csv file generated using above mentioned notebook to individual nodes
2. Check that your data has been added by executing `./scripts/fedbiomed_run node config config1.ini list` (node1) and `./scripts/fedbiomed_run node config config2.ini list` (node2)
3. Run the node using `./scripts/fedbiomed_run node config config1.ini start` (node1) and `./scripts/fedbiomed_run node config config2.ini start` (node2). Wait until you get `Starting task manager`. it means you are online.

## Create an experiment to train a model on the data found

Declare a torch training plan MyTrainingPlan class to send for training on the node

In [ ]:
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import Dataset
from fedbiomed.common.training_plans import TorchTrainingPlan
from fedbiomed.common.data import DataManager 


# Here we define the model to be used. 
# You can use any class name (here 'MyTrainingPlan')
class MyTrainingPlan(TorchTrainingPlan):
    
    # Model
    def init_model(self):
        model_args = self.model_args()
        model = self.Net(model_args)
        return model
        
    # Dependencies
    def init_dependencies(self):
        deps = ["from torch.utils.data import Dataset",
                "import pandas as pd"]
        return deps
    
    # network
    class Net(nn.Module):
        def __init__(self, model_args):
            super().__init__()
            self.in_features = model_args['in_features']
            self.out_features = model_args['out_features']
            self.fc1 = nn.Linear(self.in_features, 5)
            self.fc2 = nn.Linear(5, self.out_features)

        def forward(self, x):
            x = self.fc1(x)
            x = F.relu(x)
            x = self.fc2(x)
            return x

    def training_step(self, data, target):
        output = self.model().forward(data).float()
        criterion = torch.nn.MSELoss()
        loss   = torch.sqrt(criterion(output, target.unsqueeze(1)))
        return loss

    class csv_Dataset(Dataset):
    # Here we define a custom Dataset class inherited from the general torch Dataset class
    # This class takes as argument a .csv file path and creates a torch Dataset 
        def __init__(self, dataset_path, x_dim):
            self.input_file = pd.read_csv(dataset_path,sep=',',index_col=False)
            x_train = self.input_file.loc[:,('year','transmission','mileage','tax','mpg','engineSize')].values
            y_train = self.input_file.loc[:,'price'].values
            self.X_train = torch.from_numpy(x_train).float()
            self.Y_train = torch.from_numpy(y_train).float()

        def __len__(self):            
            return len(self.Y_train)

        def __getitem__(self, idx):

            return (self.X_train[idx], self.Y_train[idx])
        
    def training_data(self,  batch_size = 48):
    # The training_data creates the Dataloader to be used for training in the general class TorchTrainingPlan of fedbiomed
        dataset = self.csv_Dataset(self.dataset_path, self.model_args()["in_features"])
        train_kwargs = {'batch_size': batch_size, 'shuffle': True}
        return DataManager(dataset=dataset , **train_kwargs)
    
    

In [ ]:
# model parameters 
model_args = {
    'in_features': 6, 
    'out_features': 1
}

# training parameters 
training_args = {
    'batch_size': 40, 
    'optimizer_args': {
          'lr': 1e-3
    },
    'epochs': 2, 
    'dry_run': False
}

Define an experiment
- search nodes serving data for these `tags`, optionally filter on a list of node ID with `nodes`
- run a round of local training on nodes with model defined in `model_path` + federation with `aggregator`
- run for `round_limit` rounds, applying the `node_selection_strategy` between the rounds

In [ ]:
from fedbiomed.researcher.experiment import Experiment
from fedbiomed.researcher.aggregators.fedavg import FedAverage

# Calling the training data with specified tags. Change the following tag accordingly
tags =  ['UsedCars']
rounds = 3

exp = Experiment(tags=tags,
                 training_plan_class=MyTrainingPlan,
                 model_args=model_args,
                 training_args=training_args,
                 round_limit=rounds,
                 aggregator=FedAverage(),
                 node_selection_strategy=None)

Let's start the experiment.

By default, this function doesn't stop until all the `round_limit` rounds are done for all the nodes

In [ ]:
exp.run()

Local training results for each round and each node are available via `exp.training_replies()` (index 0 to (`rounds` - 1) ).

For example you can view the training results for the last round below.

Different timings (in seconds) are reported for each dataset of a node participating in a round :
- `rtime_training` real time (clock time) spent in the training function on the node
- `ptime_training` process time (user and system CPU) spent in the training function on the node
- `rtime_total` real time (clock time) spent in the researcher between sending the request and handling the response, at the `Job()` layer

In [ ]:
print("\nList the training rounds : ", exp.training_replies().keys())

print("\nList the nodes for the last training round and their timings : ")
round_data = exp.training_replies()[rounds - 1].data()
for c in range(len(round_data)):
    print("\t- {id} :\
    \n\t\trtime_training={rtraining:.2f} seconds\
    \n\t\tptime_training={ptraining:.2f} seconds\
    \n\t\trtime_total={rtotal:.2f} seconds".format(id = round_data[c]['node_id'],
        rtraining = round_data[c]['timing']['rtime_training'],
        ptraining = round_data[c]['timing']['ptime_training'],
        rtotal = round_data[c]['timing']['rtime_total']))
print('\n')
    
exp.training_replies()[rounds - 1].dataframe()

Federated parameters for each round are available via `exp.aggregated_params()` (index 0 to (`rounds` - 1) ).

For example you can view the federated parameters for the last round of the experiment :

In [ ]:
print("\nList the training rounds : ", exp.aggregated_params().keys())

print("\nAccess the federated params for the last training round :")
print("\t- params_path: ", exp.aggregated_params()[rounds - 1]['params_path'])
print("\t- parameter data: ", exp.aggregated_params()[rounds - 1]['params'].keys())


# Test Function

In [ ]:
fed_model = exp.training_plan().model()
fed_model.load_state_dict(exp.aggregated_params()[rounds - 1]['params'])

In [ ]:
 fed_model

In [ ]:
import os
# Hold one file for testing the fed model
test_dataset_path = os.path.join(data_dir, "ford_transformed.csv")

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import pandas as pd

def cal_rmse(actual, prediction):
    return ((actual- prediction)**2).mean()**0.5

def testing_rmse(model, data_loader):
    model.eval()
    test_loss = 0
    correct = 0
    device = 'cpu'
    preds = []
    with torch.no_grad():
        for data, target in data_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            preds.append(output.numpy().flatten())
    rmse = cal_rmse(data_loader.dataset.Y_train.numpy(),np.hstack(preds))
    return rmse

In [ ]:
class csv_Dataset(Dataset):
        def __init__(self, dataset_path):
            self.input_file = pd.read_csv(dataset_path,sep=',',index_col=False)
            x_train = self.input_file.loc[:,('year','transmission','mileage','tax','mpg','engineSize')].values
            y_train = self.input_file.loc[:,'price'].values
            self.X_train = torch.from_numpy(x_train).float()
            self.Y_train = torch.from_numpy(y_train).float()

        def __len__(self):            
            return len(self.Y_train)

        def __getitem__(self, idx):

            return (self.X_train[idx], self.Y_train[idx])

In [ ]:
dataset = csv_Dataset(test_dataset_path)
train_kwargs = {'batch_size': 64, 'shuffle': True}
data_loader = DataLoader(dataset, **train_kwargs)

In [ ]:
rmse = testing_rmse(fed_model, data_loader)

In [ ]:
rmse

2022-11-16 18:01:29,166 fedbiomed INFO - WARNING
					 NODE node_23b5af4d-1181-4628-8329-0067ee8030ae
					 MESSAGE: There is no validation activated for the round. Please set flag for `test_on_global_updates`, `test_on_local_updates`, or both. Splitting dataset for validation will be ignored
-----------------------------------------------------------------
2022-11-16 18:01:29,183 fedbiomed INFO - WARNING
					 NODE node_36f153c3-5f27-468c-a24e-4991a618f797
					 MESSAGE: There is no validation activated for the round. Please set flag for `test_on_global_updates`, `test_on_local_updates`, or both. Splitting dataset for validation will be ignored
-----------------------------------------------------------------
2022-11-16 18:01:29,394 fedbiomed INFO - TRAINING 
					 NODE_ID: node_23b5af4d-1181-4628-8329-0067ee8030ae 
					 Epoch: 1 | Completed: 480/60000 (1%) 
 					 Loss: 14183.883789 
					 ---------
2022-11-16 18:01:29,547 fedbiomed INFO - TRAINING 
					 NODE_ID: node_23b5af4d-1181

2022-11-16 18:01:40,416 fedbiomed INFO - TRAINING 
					 NODE_ID: node_23b5af4d-1181-4628-8329-0067ee8030ae 
					 Epoch: 1 | Completed: 960/60000 (2%) 
 					 Loss: 8549.171875 
					 ---------
2022-11-16 18:01:40,450 fedbiomed INFO - TRAINING 
					 NODE_ID: node_36f153c3-5f27-468c-a24e-4991a618f797 
					 Epoch: 1 | Completed: 3360/60000 (6%) 
 					 Loss: 7996.053711 
					 ---------
2022-11-16 18:01:40,643 fedbiomed INFO - TRAINING 
					 NODE_ID: node_36f153c3-5f27-468c-a24e-4991a618f797 
					 Epoch: 1 | Completed: 3840/60000 (6%) 
 					 Loss: 7572.130859 
					 ---------
2022-11-16 18:01:40,730 fedbiomed INFO - TRAINING 
					 NODE_ID: node_23b5af4d-1181-4628-8329-0067ee8030ae 
					 Epoch: 1 | Completed: 1440/60000 (2%) 
 					 Loss: 8456.580078 
					 ---------
2022-11-16 18:01:40,864 fedbiomed INFO - TRAINING 
					 NODE_ID: node_23b5af4d-1181-4628-8329-0067ee8030ae 
					 Epoch: 1 | Completed: 1920/60000 (3%) 
 					 Loss: 8686.101562 
					 ---------
2022-11-16 18:01:41,0

2022-11-16 18:01:51,464 fedbiomed INFO - TRAINING 
					 NODE_ID: node_36f153c3-5f27-468c-a24e-4991a618f797 
					 Epoch: 1 | Completed: 2880/60000 (5%) 
 					 Loss: 7484.039551 
					 ---------
2022-11-16 18:01:51,572 fedbiomed INFO - TRAINING 
					 NODE_ID: node_36f153c3-5f27-468c-a24e-4991a618f797 
					 Epoch: 1 | Completed: 3360/60000 (6%) 
 					 Loss: 7083.143555 
					 ---------
2022-11-16 18:01:51,680 fedbiomed INFO - TRAINING 
					 NODE_ID: node_36f153c3-5f27-468c-a24e-4991a618f797 
					 Epoch: 1 | Completed: 3840/60000 (6%) 
 					 Loss: 6744.541992 
					 ---------
2022-11-16 18:01:51,785 fedbiomed INFO - TRAINING 
					 NODE_ID: node_36f153c3-5f27-468c-a24e-4991a618f797 
					 Epoch: 1 | Completed: 4320/60000 (7%) 
 					 Loss: 6849.343750 
					 ---------
2022-11-16 18:01:51,892 fedbiomed INFO - TRAINING 
					 NODE_ID: node_36f153c3-5f27-468c-a24e-4991a618f797 
					 Epoch: 1 | Completed: 4800/60000 (8%) 
 					 Loss: 6741.793945 
					 ---------
2022-11-16 18:01:51,

2022-11-16 18:11:25,412 fedbiomed INFO - TRAINING 
					 NODE_ID: node_36f153c3-5f27-468c-a24e-4991a618f797 
					 Epoch: 0 | Completed: 100/150 (67%) 
 					 Loss perceptron: 0.000000 
					 ---------
2022-11-16 18:11:25,413 fedbiomed INFO - INFO
					 NODE node_36f153c3-5f27-468c-a24e-4991a618f797
					 MESSAGE: Reached 100 batches for this epoch, ignore remaining data
-----------------------------------------------------------------
2022-11-16 18:11:25,416 fedbiomed INFO - TRAINING 
					 NODE_ID: node_23b5af4d-1181-4628-8329-0067ee8030ae 
					 Epoch: 0 | Completed: 70/100 (70%) 
 					 Loss perceptron: 0.000000 
					 ---------
2022-11-16 18:11:25,420 fedbiomed INFO - TRAINING 
					 NODE_ID: node_36f153c3-5f27-468c-a24e-4991a618f797 
					 Epoch: 1 | Completed: 10/150 (7%) 
 					 Loss perceptron: 0.000000 
					 ---------
2022-11-16 18:11:25,425 fedbiomed INFO - TRAINING 
					 NODE_ID: node_23b5af4d-1181-4628-8329-0067ee8030ae 
					 Epoch: 0 | Completed: 80/100 (80%) 
 					 L

2022-11-16 18:11:25,664 fedbiomed INFO - TRAINING 
					 NODE_ID: node_36f153c3-5f27-468c-a24e-4991a618f797 
					 Epoch: 2 | Completed: 80/150 (53%) 
 					 Loss perceptron: 0.000000 
					 ---------
2022-11-16 18:11:25,669 fedbiomed INFO - TRAINING 
					 NODE_ID: node_36f153c3-5f27-468c-a24e-4991a618f797 
					 Epoch: 2 | Completed: 90/150 (60%) 
 					 Loss perceptron: 0.000000 
					 ---------
2022-11-16 18:11:25,672 fedbiomed INFO - TRAINING 
					 NODE_ID: node_23b5af4d-1181-4628-8329-0067ee8030ae 
					 Epoch: 2 | Completed: 20/100 (20%) 
 					 Loss perceptron: 0.000000 
					 ---------
2022-11-16 18:11:25,675 fedbiomed INFO - TRAINING 
					 NODE_ID: node_36f153c3-5f27-468c-a24e-4991a618f797 
					 Epoch: 2 | Completed: 100/150 (67%) 
 					 Loss perceptron: 0.000000 
					 ---------
2022-11-16 18:11:25,684 fedbiomed INFO - INFO
					 NODE node_36f153c3-5f27-468c-a24e-4991a618f797
					 MESSAGE: Reached 100 batches for this epoch, ignore remaining data
-----------------------

2022-11-16 18:11:25,840 fedbiomed INFO - TRAINING 
					 NODE_ID: node_36f153c3-5f27-468c-a24e-4991a618f797 
					 Epoch: 4 | Completed: 90/150 (60%) 
 					 Loss perceptron: 110.553336 
					 ---------
2022-11-16 18:11:25,841 fedbiomed INFO - TRAINING 
					 NODE_ID: node_23b5af4d-1181-4628-8329-0067ee8030ae 
					 Epoch: 3 | Completed: 60/100 (60%) 
 					 Loss perceptron: 0.000000 
					 ---------
2022-11-16 18:11:25,846 fedbiomed INFO - TRAINING 
					 NODE_ID: node_36f153c3-5f27-468c-a24e-4991a618f797 
					 Epoch: 4 | Completed: 100/150 (67%) 
 					 Loss perceptron: 14.288779 
					 ---------
2022-11-16 18:11:25,848 fedbiomed INFO - INFO
					 NODE node_36f153c3-5f27-468c-a24e-4991a618f797
					 MESSAGE: Reached 100 batches for this epoch, ignore remaining data
-----------------------------------------------------------------
2022-11-16 18:11:25,849 fedbiomed INFO - TRAINING 
					 NODE_ID: node_23b5af4d-1181-4628-8329-0067ee8030ae 
					 Epoch: 3 | Completed: 70/100 (70%) 
 			

2022-11-16 18:11:35,183 fedbiomed INFO - TRAINING 
					 NODE_ID: node_36f153c3-5f27-468c-a24e-4991a618f797 
					 Epoch: 0 | Completed: 30/150 (20%) 
 					 Loss perceptron: 0.000000 
					 ---------
2022-11-16 18:11:35,187 fedbiomed INFO - TRAINING 
					 NODE_ID: node_23b5af4d-1181-4628-8329-0067ee8030ae 
					 Epoch: 0 | Completed: 30/100 (30%) 
 					 Loss perceptron: 251.634093 
					 ---------
2022-11-16 18:11:35,191 fedbiomed INFO - TRAINING 
					 NODE_ID: node_36f153c3-5f27-468c-a24e-4991a618f797 
					 Epoch: 0 | Completed: 40/150 (27%) 
 					 Loss perceptron: 0.000000 
					 ---------
2022-11-16 18:11:35,198 fedbiomed INFO - TRAINING 
					 NODE_ID: node_23b5af4d-1181-4628-8329-0067ee8030ae 
					 Epoch: 0 | Completed: 40/100 (40%) 
 					 Loss perceptron: 99.596415 
					 ---------
2022-11-16 18:11:35,204 fedbiomed INFO - TRAINING 
					 NODE_ID: node_36f153c3-5f27-468c-a24e-4991a618f797 
					 Epoch: 0 | Completed: 50/150 (33%) 
 					 Loss perceptron: 0.000000 
					 --

2022-11-16 18:11:35,322 fedbiomed INFO - TRAINING 
					 NODE_ID: node_23b5af4d-1181-4628-8329-0067ee8030ae 
					 Epoch: 1 | Completed: 70/100 (70%) 
 					 Loss perceptron: 0.000000 
					 ---------
2022-11-16 18:11:35,325 fedbiomed INFO - TRAINING 
					 NODE_ID: node_36f153c3-5f27-468c-a24e-4991a618f797 
					 Epoch: 2 | Completed: 40/150 (27%) 
 					 Loss perceptron: 0.000000 
					 ---------
2022-11-16 18:11:35,327 fedbiomed INFO - TRAINING 
					 NODE_ID: node_36f153c3-5f27-468c-a24e-4991a618f797 
					 Epoch: 2 | Completed: 50/150 (33%) 
 					 Loss perceptron: 5.161693 
					 ---------
2022-11-16 18:11:35,329 fedbiomed INFO - TRAINING 
					 NODE_ID: node_23b5af4d-1181-4628-8329-0067ee8030ae 
					 Epoch: 1 | Completed: 80/100 (80%) 
 					 Loss perceptron: 0.000000 
					 ---------
2022-11-16 18:11:35,332 fedbiomed INFO - TRAINING 
					 NODE_ID: node_36f153c3-5f27-468c-a24e-4991a618f797 
					 Epoch: 2 | Completed: 60/150 (40%) 
 					 Loss perceptron: 0.000000 
					 -----

2022-11-16 18:11:35,450 fedbiomed INFO - TRAINING 
					 NODE_ID: node_36f153c3-5f27-468c-a24e-4991a618f797 
					 Epoch: 4 | Completed: 40/150 (27%) 
 					 Loss perceptron: 0.000000 
					 ---------
2022-11-16 18:11:35,454 fedbiomed INFO - TRAINING 
					 NODE_ID: node_36f153c3-5f27-468c-a24e-4991a618f797 
					 Epoch: 4 | Completed: 50/150 (33%) 
 					 Loss perceptron: 0.000000 
					 ---------
2022-11-16 18:11:35,457 fedbiomed INFO - TRAINING 
					 NODE_ID: node_23b5af4d-1181-4628-8329-0067ee8030ae 
					 Epoch: 3 | Completed: 10/100 (10%) 
 					 Loss perceptron: 0.000000 
					 ---------
2022-11-16 18:11:35,461 fedbiomed INFO - TRAINING 
					 NODE_ID: node_36f153c3-5f27-468c-a24e-4991a618f797 
					 Epoch: 4 | Completed: 60/150 (40%) 
 					 Loss perceptron: 0.105358 
					 ---------
2022-11-16 18:11:35,467 fedbiomed INFO - TRAINING 
					 NODE_ID: node_36f153c3-5f27-468c-a24e-4991a618f797 
					 Epoch: 4 | Completed: 70/150 (47%) 
 					 Loss perceptron: 0.000000 
					 -----